In [237]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
#from IPython.core.display import display, HTML

In [239]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [253]:
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity


In [255]:
!pip install wordcloud
from bokeh.io import show,curdoc,output_notebook,push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource,HoverTool,Select,Paragraph,TextInput
from ipywidgets import interact
from sklearn.metrics.pairwise import linear_kernel
!pip install markovify
import markovify
import string
from os import path
from PIL import Image
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from sklearn.pipeline import Pipeline



In [256]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

In [257]:
import warnings; warnings.simplefilter('ignore')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [261]:
from sklearn.metrics import classification_report

In [263]:
df=pd.read_csv('skindataall.csv')

In [265]:
df

,Unnamed: 0,Username,Skin_Tone,Skin_Type,Eye_Color,Hair_Color,Rating_Stars,Review,Product,Brand,...,Oily,Sensitive,Category,Product_Url,User_id,Product_id,Ingredients_Cleaned,Review_Cleaned,Good_Stuff,Ing_Tfidf
0,0,allyp3,Medium,Combination,Brown,Brunette,5,"This is hands down the best cleanser I’ve ever used. I originally got it in the mini bottle at checkout on a whim and was shocked by how much it cleared my skin up. I bought the big bottle and haven’t looked back since, now this is the only cleanser I’ll use. I used to get little pimples all over my forehead and ever since I started using this (a year ago) they went away and haven’t returned. While I can’t speak for everyone’s skin, this was a miracle product for clearing my complexion and soothes my facial redness each time I use it. Im in love.\n…read more",Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,...,0,0,Cleanser,https://www.sephora.com/product/kale-spinach-green-tea-age-prevention-cleanser-P411387?icid2=products%20grid:p411387,3420,157,"['sodium', 'cocoyl', 'glutamate', 'cocamidopropyl', 'betaine', 'panthenol', 'vitamin', 'b', 'tocopheryl', 'acetate', 'vitamin', 'e', 'ascorbic', 'acid', 'vitamin', 'c', 'butylene', 'glycol', 'brassica', 'oleracea', 'kale', 'leaf', 'extract', 'spinacia', 'oleracea', 'spinach', 'leaf', 'extract', 'camellia', 'sinensis', 'green', 'tea', 'leaf', 'extract', 'chamomilla', 'recutita', 'matricaria', 'flower', 'extract', 'medicago', 'sativa', 'alfalfa', 'extract', 'aloe', 'barbadensis', 'aloe', 'vera', 'leaf', 'extract', 'natural', 'fragrance', 'lecithin', 'hydroxypropylmethylcellulose', 'phenoxyethanol', 'potassium', 'sorbate', 'sodium', 'benzoate', 'chlorophyll']","['hand', 'best', 'cleanser', 'ever', 'used', 'originally', 'got', 'mini', 'bottle', 'checkout', 'whim', 'wa', 'shocked', 'much', 'cleared', 'skin', 'bought', 'big', 'bottle', 'looked', 'back', 'since', 'cleanser', 'use', 'used', 'get', 'little', 'pimple', 'forehead', 'ever', 'since', 'started', 'using', 'year', 'ago', 'went', 'away', 'returned', 'speak', 'everyone', 'skin', 'wa', 'miracle', 'clearing', 'complexion', 'soothes', 'facial', 'redness', 'time', 'use', 'im', 'love', 'read']",1,"sodium, cocoyl, glutamate, cocamidopropyl, betaine, panthenol, vitamin, b, tocopheryl, acetate, vitamin, e, ascorbic, acid, vitamin, c, butylene, glycol, brassica, oleracea, kale, leaf, extract, spinacia, oleracea, spinach, leaf, extract, camellia, sinensis, green, tea, leaf, extract, chamomilla, recutita, matricaria, flower, extract, medicago, sativa, alfalfa, extract, aloe, barbadensis, aloe, vera, leaf, extract, natural, fragrance, lecithin, hydroxypropylmethylcellulose, phenoxyethanol, potassium, sorbate, sodium, benzoate, chlorophyll"
1,1,PatTea,Medium,Combination,Brown,Red,1,"Unfortunately this doesn’t work for everyone. I used it once and woke up the nxt day with 5 pimples, used it again, and again, and again but woke up with more and more. Finally i couldn’t take it anymore, I stopped using it, went back to my cerave hydrating cleanser- and Bam! My skin was clearing up every day after that. I suggest getting the small bottle to try it out first. Remember everyone’s skin is very different.\n…read more",Superfood Antioxidant Cleanser,YOUTH TO THE PEOPLE,...,0,0,Cleanser,https://www.sephora.com/product/kale-spinach-green-tea-age-prevention-cleanser-P411387?icid2=products%20grid:p411387,2483,157,"['sodium', 'cocoyl', 'glutamate', 'cocamidopropyl', 'betaine', 'panthenol', 'vitamin', 'b', 'tocopheryl', 'acetate', 'vitamin', 'e', 'ascorbic', 'acid', 'vitamin', 'c', 'butylene', 'glycol', 'brassica', 'oleracea', 'kale', 'leaf', 'extract', 'spinacia', 'oleracea', 'spinach', 'leaf', 'extract', 'camellia', 'sinensis', 'green', 'tea', 'leaf', 'extract', 'chamomilla', 'recutita', 'matricaria', 'flower', 'extract', 'medicago', 'sativa', 'alfalfa', 'extract', 'aloe', 'barbadensis', 'aloe', 'vera', 'leaf', 'extract', 'natural', 'fragrance', 'lecithin', 'hydroxypropylmethylcellulose', 'phenoxyethano

In [267]:
def no_punc(string):
    no_punc_string=re.sub('[^\w\s]','',string)
    return no_punc_string

In [269]:
df['Review']=df.apply(lambda row: no_punc(row['Review']),axis=1)
print(df['Review'].head())


0                                                                                                                                                             This is hands down the best cleanser Ive ever used I originally got it in the mini bottle at checkout on a whim and was shocked by how much it cleared my skin up I bought the big bottle and havent looked back since now this is the only cleanser Ill use I used to get little pimples all over my forehead and ever since I started using this a year ago they went away and havent returned While I cant speak for everyones skin this was a miracle product for clearing my complexion and soothes my facial redness each time I use it Im in love\nread more
1                                                                                                                                                                                                                                                                                              Unfortu

In [271]:
text_model=markovify.NewlineText(df.Review,state_size=2)

In [272]:
negativedf=df[df.Rating_Stars<=4]
positvedf=df[df.Rating_Stars>4]

In [273]:
negative=" ".join(review for review in negativedf.Review)


In [275]:
positive=" ".join(review for review in positvedf.Review)

In [279]:
stopwords=set(STOPWORDS)
stopwords.update(['read','more','product'])

In [281]:
def generate_wordcloud(data):
    wordcloud=WordCloud(stopwords=stopwords,background_color='white').generate(data)
    plt.imshow(wordcloud,interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [283]:
df['Category'].value_counts()
moist_df=df[df.Category=='Moisturizer']
cleanse_df=df[df.Category=='Cleanser']
mask_df=df[df.Category=='Face Mask']
treat_df=df[df.Category=='Treatment']
moist=" ".join(ing for ing in moist_df.Ingredients)
cleanse=" ".join(ing for ing in cleanse_df.Ingredients )
mask=" ".join(ing for ing in mask_df.Ingredients)
treat=" ".join(ing for ing in treat_df.Ingredients)

In [285]:
x=df.Ingredients

In [287]:
y=df.Category

In [289]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=42)
categories=['Moisturizer','Cleanser','Face mask','Treatment']

In [291]:
nb=Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',MultinomialNB())])
nb.fit(xtrain,ytrain)
y_pred=nb.predict(xtest)

In [292]:
logreg=Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',LogisticRegression(n_jobs=1,C=1e5))])
logreg.fit(xtrain,ytrain)
y_pred=logreg.predict(xtest)
                 

                

In [294]:
sgd=Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',SGDClassifier(loss='hinge',penalty='l2',alpha=1e-3,random_state=42,max_iter=5))])
sgd.fit(xtrain,ytrain)
ypred=sgd.predict(xtest)

In [295]:
x1=df.Ingredients
y1=df.Good_Stuff

In [297]:
xtrain1,xtest1,ytrain1,ytest1=train_test_split(x1,y1,test_size=25,random_state=42)
cat=['1','0']

In [298]:
nb=Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',MultinomialNB())])
nb.fit(xtrain1,ytrain1)
y_pred=nb.predict(xtest1)


In [302]:
logreg=Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',LogisticRegression(n_jobs=1,C=1e5))])
logreg.fit(xtrain1,ytrain1)
y_pred1=logreg.predict(xtest1)

In [303]:
sgd=Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',SGDClassifier(loss='hinge',penalty='l2',alpha=1e-3,random_state=42,max_iter=5))])
sgd.fit(xtrain1,ytrain1)
y_pred1=sgd.predict(xtest1)


In [305]:
def recommend_products_by_user_features(skintone,eyecolor,skintype,haircolor):
        ddf=df[(df['Skin_Tone']==skintone) & (df['Eye_Color']==eyecolor) & (df['Skin_Type']==skintype) & (df['Hair_Color']==haircolor)]
        recommendations=ddf[(ddf['Rating_Stars'].notnull())][['Rating_Stars','Product_Url','Product','Price','Ing_Tfidf']]
        recommendations=recommendations.sort_values('Rating_Stars',ascending=False).head(5)
        print('Based on your features,these are the top products for you')
        return recommendations

In [306]:
skintone="Light"
eyecolor="Green"
skintype="Combination"
haircolor="Brunette"


In [308]:
df_cont=df[['Product','Product_id','Ingredients','Product_Url','Ing_Tfidf','Rating','Price']]
df_cont.drop_duplicates(inplace=True)
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=1,stop_words='english')

In [309]:
tfidf_matrix=tf.fit_transform(df_cont['Ingredients'])
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)
df_cont.reset_index(drop=True)
titles=df_cont[['Product','Ing_Tfidf','Rating','Price','Product_Url']]
indices=pd.Series(df_cont.index,index=df_cont['Product'])

In [313]:
def content_recommendation(product):
    idx=indices[product]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores=sorted(sim_scores,key=lambda x:x[1], reverse=True)
    sim_scores=sim_scores[1:11]
    product_indices=[i[0] for i in sim_scores]
    print(titles)
    recommendations = titles.iloc[product_indices][['Product', 'Rating', 'Price', 'Product_Url']]
    recommendations.reset_index(drop=True, inplace=True)
    #recs = titles.iloc[product_indices][['Product', 'Rating', 'Price', 'Product_Url']].copy()
    #recs.reset_index(drop=True, inplace=True)
    #recs['Product_Url'] = recs['Product_Url'].apply(lambda url: f"<a href='{url}' target='_blank'>Link</a>")
    #display(HTML(recs.to_html(escape=False, index=False)))
    return recommendations
    return titles.iloc[product_indices]

In [317]:
recommend_products_by_user_features("Light", "Green", "Combination", "Brunette")



Based on your features,these are the top products for you


,Rating_Stars,Product_Url,Product,Price,Ing_Tfidf
349,5,https://www.sephora.com/product/bi-facil-double-action-eye-makeup-remover-P54477?icid2=products%20grid:p54477,Bi-Facil Double-Action Eye Makeup Remover,30,"cyclopentasiloxane, isohexadecane, sodium, chloride, poloxamer, hexylene, glycol, dipotassium, phosphate, benzyl, alochol, potassium, phosphate, quaternium, benzalkonium, chloride, fragrance, citronellol, geraniol"
1975,5,https://www.sephora.com/product/ultimate-miracle-worker-multi-rejuvenating-cream-spf-30-P398831?icid2=products%20grid:p398831,Ultimate Miracle Worker Multi-Rejuvenating Cream SPF 30,75,"c, alkyl, benzoate, carthamus, tinctorius, safflower, oleosomes, glycerin, cyclopentasiloxane, butylene, glycol, methyl, gluceth, gluconolactone, polyacrylamide, hydrogenated, lecithin, cyclohexasiloxane, cetyl, alcohol, bifida, ferment, lysate, phenoxyethanol, ammonium, acryloyldimethyltaurate, vp, copolymer, c, isoparaffin, polysorbate, acrylate, c, alkyl, acrylate, crosspolymer, sodium, benzoate, sodium, hydroxide, laureth, glycine, soja, soybean, oil, fragrance, iris, pallida, root, extract, ruby, powder, disodium, edta, gluconic, acid, alcohol, mica, lecithin, adenosine, cellulose, gum, retinol, peg, polysorbate, bht, chitosan, ethylhexylglycerin, glycolic, acid, sorbitan, oleate, calcium, gluconate, tocopherol, behentrimonium, chloride, sodium, bicarbonate, ascorbyl, palmitate, beta, glucan, potassium, phosphate, caprylyl, glycol, hexanediol, camellia, sinensis, leaf, extract, coffea, arabica, coffee, seed, extract, pongamia, pinnata, seed, extract, isopropyl, alcohol, acetic, acid, angelica, archangelica, root, extract, benzoic, acid, citrus, aurantium, amara, bitter, orange, peel, extract, maltodextrin, citric, acid, magnesium, aluminum, silicate, xanthan, gum, lactic, acid, ascorbic, acid, sclerotium, gum, bha, sorbic, acid, titanium, dioxide, ci"
2796,5,https://www.sephora.com/product/aqua-bomb-sleeping-mask-P433443?icid2=products%20grid:p433443,Aqua Bomb Sleeping Mask,34,"dipropylene, glycol, polyglycerin, hexanediol, glycerin, peg, hdi, copolymer, bi, decyltetradecet, ether, polymethylsilsequioxane, calluna, vulgaris, extract, dispacus, sylvestris, extract, crysanthemum, parthenium, feverfew, extract, avena, sative, oat, kernel, extract, calendula, officinalis, flower, extract, nepeta, cataria, extract, rubus, idaeus, raspberry, leaf, extract, baptisia, tinctoria, root, extract, stellaria, medium, chickweed, extract, malt, extract, triticum, vulgare, wheat, sprout, extract, alchemilla, vulgaris, leaf, extract, equisetum, arvense, leaf, extract, urticadioicia, nettle, leaf, extract, melia, azadirachta, leaf, extract, melia, azadirachta, flower, extract, coccinia, indica, fruit, extract, solanum, melongena, eggplant, fruit, extract, aloe, barbadensis, flower, extract, simmondsia, chinesis, jojoba, seed, oil, ocimum, sanctum, leaf, extract, curcuma, longa, tumeric, root, extract, corallina, officinalis, extract, panthenol, tocopheryl, acetate, lauryl, dimethicone, polyglycerin, crosspolymer, acrylate, c, alkyl, acrylate, crosspolymer, carbomer, hydroxyethylcellulose, trisodium, edta, tromethamine, citrus, aurantium, dulcis, orange, peel, oil, citrus, aurantifolia, lime, oil, pelargonium, graveolens, flower, oil, rosmarinus, officinalis, rosemary, leaf, oil, citral, citronellol, geraniol, limonene, linalool, napier, original, formula, napier, formula, fragrance"
2914,5,https://www.sephora.com/product/vitamin-c-antioxidant-glow-face-mist-P433979?icid2=products%20grid:p433979,Vitamin Nectar Antioxidant Face Mist,44,"glycerin, penty, lene, glycol, peg, hydrogenated, castor, oil, vaccinium, myrtillus, fruit, extract, saccharum, officinarum, sugar, cane, extract, citrus, au, rantium, dulcis, orange, fruit, extract, citrus, limon, lemon, fruit, extract, acer, saccharum, sugar, maple, extract, tocopheryl, acetate, ascorbyl, tetraisopalmitate, sodium, tocopheryl, phosphate, tocopherol, panthenol, magnesium, aspartate, 

In [161]:
content_recommendation('The Rice Polish Foaming Enzyme Powder')


                                                 Product  \
0                         Superfood Antioxidant Cleanser   
48                         Cleansing & Exfoliating Wipes   
96                           Purity Made Simple Cleanser   
144                          Beste™ No. 9 Jelly Cleanser   
192                The Rice Polish Foaming Enzyme Powder   
...                                                  ...   
8533       Midnight Secret Late Night Recovery Treatment   
8556                       Post-Acne Spot Lightening Gel   
8580     Hydro-Plumping Re-Texturizing Serum Concentrate   
8603                  The Microdelivery Resurfacing Peel   
8626  Ferulic + Retinol Wrinkle Recovery Overnight Serum   

                                                                                                                                                                                                                                                                                       

,Product,Rating,Price,Product_Url
0,Water Drench Hyaluronic Cloud Serum,4.2,65,https://www.sephora.com/product/water-drench-hyaluronic-cloud-serum-P222818?icid2=products%20grid:p222818
1,Water Drench Hyaluronic Cloud Cream,4.4,52,https://www.sephora.com/product/water-drench-hyaluronic-cloud-cream-P415701?icid2=products%20grid:p415701
2,Nutrient-Charged Water Gel,4.6,60,https://www.sephora.com/product/nutrient-charged-water-gel-P427638?icid2=products%20grid:p427638
3,Problem Solution Toner,4.3,28,https://www.sephora.com/product/problem-solution-toner-P410817?icid2=products%20grid:p410817
4,The True Cream Moisturizing Bomb,4.6,38,https://www.sephora.com/product/the-true-cream-moisturizing-bomb-P394624?icid2=products%20grid:p394624
5,Skin Rescue Acne Clearing Pads with White Clay,4.2,30,https://www.sephora.com/product/skin-rescue-acne-clearing-pads-with-white-clay-P430140?icid2=products%20grid:p430140
6,Deep Cleansing Exfoliator,4.6,35,https://www.sephora.com/product/deep-cleansing-exfoliator-P374574?icid2=products%20grid:p374574
7,Treatment Toner,4.4,50,https://www.sephora.com/product/treatment-toner-P232934?icid2=products%20grid:p232934
8,Superberry Hydrate + Glow Oil,4.8,44,https://www.sephora.com/product/superberry-hydrate-glow-oil-P430146?icid2=products%20grid:p430146
9,Goodnight Glow Retin-ALT Sleeping Crème,4.3,55,https://www.sephora.com/product/goodnight-glow-retin-alt-sleeping-creme-P434546?icid2=products%20grid:p434546
